Diego Toribio <br>
ECE-465: Cloud Computing <br>
Professor Marano <br>
Problem Set 1: Dining Philosphers Problem

# Instructions

You are to provide a solution to the Dining Philosophers problem. It's a classic illustration of concurrency and some of its fundamental challenges like **deadlock** and **starvation**.

Imagine a scenario with five philosophers seated around a circular table. Between each pair of adjacent philosophers lies a single chopstick. Each philosopher alternates between **thinking** and **eating**. To eat, a philosopher needs two chopsticks – the ones to their immediate left and right.

The problem arises because if all philosophers simultaneously decide to eat and each grabs their left chopstick, they'll all be stuck. No one can eat because everyone is waiting for the chopstick held by their neighbor. This is a classic **deadlock** situation: everyone is blocked, waiting for a resource that will never become available.

<br>

Here's a breakdown of the key elements and challenges:
- **Philosophers:** Represent processes or threads competing for resources.  
- **Chopsticks:** Represent shared resources. They are a limited quantity.  
- **Eating:** Represents a process holding multiple resources (two chopsticks) to perform some work.  
- **Thinking:** Represents a process releasing resources and not needing them.  

<br>
The Dining Philosophers problem highlights several important concurrency concepts:

- **Deadlock:** A situation where two or more processes are blocked indefinitely, waiting for each other to release resources.  
- **Starvation:** A situation where a process is repeatedly denied access to a resource, even though the resource is periodically available. For example, one philosopher might consistently be unlucky and never get both chopsticks, even if deadlock is avoided.  
- **Mutual Exclusion:** Only one philosopher can use a chopstick at a time. This is a fundamental requirement for shared resources.  


In [11]:
import threading
import time
import random
from tabulate import tabulate

## Approach

To solve the Dining Philosophers problem, we use a *multi-threading* approach with **ordered chopstick acquisition** to prevent deadlock. The simulation mimics a circular arrangement of philosophers and chopsticks, where each philosopher must secure two chopsticks (shared resources) to eat.

### Circular Arrangement and Chopstick Numbering

- **Circular Table:**  
  Imagine 5 philosophers sitting around a circular table. A chopstick is placed between each pair of adjacent philosophers.

- **Chopstick Numbering:**  
  With 5 philosophers, there are 5 chopsticks. We label these chopsticks 0 through 4 in a clockwise order. For instance:
  - **Philosopher 0:** Has chopstick 0 on their left and chopstick 1 on their right.
  - **Philosopher 1:** Has chopstick 1 on the left and chopstick 2 on the right.
  - **Philosopher 2:** Has chopstick 2 on the left and chopstick 3 on the right.
  - **Philosopher 3:** Has chopstick 3 on the left and chopstick 4 on the right.
  - **Philosopher 4:** Has chopstick 4 on the left and, because of the circular arrangement, chopstick 0 on the right.

### Ordered Resource Acquisition

- **Why Order Matters:**  
  The key to avoiding deadlock is to ensure that not all philosophers are waiting on each other in a circular chain. This is done by enforcing an order in which resources (chopsticks) are acquired.

- **Ordered Acquisition Strategy:**  
  Each philosopher picks up the lower-numbered chopstick first, then the higher-numbered one. For example:
  - **Philosopher 0:** Picks up chopstick 0 (lower) and then chopstick 1.
  - **Philosopher 1:** Picks up chopstick 1 (lower) and then chopstick 2.
  - …and so on.

- **Breaking the Circular Wait:**  
  To break the potential deadlock cycle, one philosopher (typically Philosopher 4) reverses this order. Instead of picking up the lower-numbered chopstick first (which would normally be chopstick 0 for Philosopher 4), they pick up the higher-numbered chopstick first (chopstick 4) and then chopstick 0. This inversion breaks the circular wait condition and ensures that at least one philosopher can eventually secure both chopsticks and eat.

### Concurrency Constructs

- **`threading.Thread`:**  
  Each philosopher is represented by a separate thread. Threads allow multiple philosophers (processes) to run concurrently, simulating independent actions like thinking and eating.

- **`threading.Lock`:**  
  Each chopstick is represented by a lock. A lock ensures **mutual exclusion**, meaning that only one philosopher (thread) can hold a chopstick (lock) at a time. When a philosopher acquires a lock, no other philosopher can use that chopstick until it is released.

- **State Logging:**  
  We maintain simple counters to log the state of each philosopher, such as how many times each philosopher has eaten. This helps in measuring performance, detecting starvation, and ensuring fairness.


In [3]:
NUM_PHILOSOPHERS = 5

# list of Locks, one per chopstick (chopsticks are arranged in a circle and numbered 0 to 4)
chopsticks = [threading.Lock() for _ in range(NUM_PHILOSOPHERS)]

# global arrays for performance metrics
eat_count = [0] * NUM_PHILOSOPHERS
wait_times = [0.0] * NUM_PHILOSOPHERS
attempts = [0] * NUM_PHILOSOPHERS

# shared flag to stop the simulation
stop_simulation = False


def philosopher_with_wait(phil_id: int):
    """
    Philosopher function with performance measurement.
    Measures wait time for acquiring chopsticks.
    """
    global stop_simulation
    # determine chopstick indices for the circular table:
    # Philosopher i has chopsticks i (left) and (i+1)%NUM_PHILOSOPHERS (right)
    left_chopstick = phil_id
    right_chopstick = (phil_id + 1) % NUM_PHILOSOPHERS

    # invert the order for the last philosopher to break the circular wait (deadlock)
    if phil_id == NUM_PHILOSOPHERS - 1:
        left_chopstick, right_chopstick = right_chopstick, left_chopstick

    while not stop_simulation:
        # 1. thinking (simulate with random delay)
        think_time = random.uniform(0.1, 0.5)
        time.sleep(think_time)

        # 2. attempt to acquire chopsticks (and measure wait time)
        start_wait = time.perf_counter()
        chopsticks[left_chopstick].acquire()
        chopsticks[right_chopstick].acquire()
        end_wait = time.perf_counter()

        # update performance metrics
        wait_times[phil_id] += (end_wait - start_wait)
        attempts[phil_id] += 1

        # 3. eating (simulate with random delay)
        eat_time = random.uniform(0.1, 0.5)
        eat_count[phil_id] += 1
        time.sleep(eat_time)

        # 4. release chopsticks
        chopsticks[right_chopstick].release()
        chopsticks[left_chopstick].release()


def philosopher(phil_id: int):
    """
    A simpler philosopher function without performance measurement.
    """
    global stop_simulation
    left_chopstick = phil_id
    right_chopstick = (phil_id + 1) % NUM_PHILOSOPHERS

    if phil_id == NUM_PHILOSOPHERS - 1:
        left_chopstick, right_chopstick = right_chopstick, left_chopstick

    while not stop_simulation:
        # thinking
        think_time = random.uniform(0.1, 0.5)
        time.sleep(think_time)

        # acquire chopsticks
        chopsticks[left_chopstick].acquire()
        chopsticks[right_chopstick].acquire()

        # eating
        eat_time = random.uniform(0.1, 0.5)
        eat_count[phil_id] += 1
        time.sleep(eat_time)

        # release chopsticks
        chopsticks[right_chopstick].release()
        chopsticks[left_chopstick].release()

## Running the Simulation

We'll create 5 threads (one for each philosopher), start them, let them run for a defined duration (e.g., 5 seconds), and then stop. We'll observe how many times each philosopher has eaten to see if one philosopher is starved or if the load is fairly balanced.

## Performance Measurement

To evaluate the solution’s performance, we track key concurrency metrics:

### 1. Simple Metrics
- **Eating Count:** How many times each philosopher successfully ate within the given simulation time.
- **Wait Time:** How long each philosopher waited to acquire both chopsticks.
- **Throughput:** Total successful eating events divided by the total simulation time.


### 2. Measurement Tools
- `time.perf_counter()`: Measures wait time with high precision.
- **Global Counters:** Store wait times, attempts, and eating counts

<br>
<br>


These metrics help us asssess whether:

- Deadlock is avoided (no philospher waits indefinitely).
- Starvation is minimized (philosphers eat fairly).


In [10]:
def run_simulation(run_time=5, measure_performance=True):
    """
    Runs the Dining Philosophers simulation.
    
    Parameters:
      run_time (int): Duration for which the simulation will run.
      measure_performance (bool): If True, uses the philosopher_with_wait function; otherwise, uses the simpler philosopher function.
    
    Steps:
      - Reinitialize performance metrics.
      - Create a thread for each philosopher.
      - Start all threads and run for the specified duration.
      - Signal threads to stop and wait for them to finish.
      - Return performance metrics.
    """
    global stop_simulation, eat_count, wait_times, attempts
    stop_simulation = False

    # reset global performance metrics
    for i in range(NUM_PHILOSOPHERS):
        wait_times[i] = 0.0
        attempts[i] = 0
        eat_count[i] = 0

    threads = []
    for i in range(NUM_PHILOSOPHERS):
        if measure_performance:
            t = threading.Thread(target=philosopher_with_wait, args=(i,))
        else:
            t = threading.Thread(target=philosopher, args=(i,))
        threads.append(t)

    for t in threads:
        t.start()

    time.sleep(run_time)
    stop_simulation = True

    for t in threads:
        t.join()

    return eat_count, wait_times, attempts


if __name__ == "__main__":
    # run simulation for 5 seconds
    run_simulation(run_time=5, measure_performance=True)

    # display results
    table_data = []
    for i in range(NUM_PHILOSOPHERS):
        avg_wait = wait_times[i] / attempts[i] if attempts[i] > 0 else 0
        table_data.append([i, eat_count[i], f"{avg_wait:.4f}"])

    headers = ["Philosopher", "Times Eaten", "Avg Wait Time (s)"]
    print(tabulate(table_data, headers=headers, tablefmt="pretty"))

+-------------+-------------+-------------------+
| Philosopher | Times Eaten | Avg Wait Time (s) |
+-------------+-------------+-------------------+
|      0      |      8      |      0.1970       |
|      1      |      8      |      0.1564       |
|      2      |      7      |      0.1335       |
|      3      |      8      |      0.1312       |
|      4      |      8      |      0.0540       |
+-------------+-------------+-------------------+
